# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 3.2

Implement a classifier that achieves an auc, $a>0.6$. This demonstrates that you have discovered some moderately useful features and can control how your model performs. You will not receive any extra points for a model which performs better than 0.6 in this question. 

See details on how to create the class at the end of the notebook. 

Notes:
1. See details at the end of this notebook.
1. Do not use the target label as features

In [1]:
require './final_project_lib'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

## Question 0 (0 Points)

Paste whatever of **your code** you want to use here. You should have completed them in "Final Project Tests" in this folder. 

In [2]:
# BEGIN YOUR CODE
# require './assignment_lib'

#Initializes the database used for this assignment
dir = "/home/dataset"
$dev_db = SQLite3::Database.new "#{dir}/credit_risk_data_dev.db", results_as_hash: true, readonly: true
#END YOUR CODE

#<SQLite3::Database:0x0000558558fb8b90 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @type_translator=#<Proc:0x0000558559172120 /usr/local/lib/ruby/gems/2.7.0/gems/sqlite3-1.4.2/lib/sqlite3/database.rb:722 (lambda)>, @readonly=true>

In [3]:
def mean x
  # BEGIN YOUR CODE
  x = x.compact
  x.inject(0.0){|sum,i| sum + i }/x.length()  
  #END YOUR CODE
end

:mean

In [4]:
def stdev x
  # BEGIN YOUR CODE
  x = x.compact
  mu = mean x
  su = x.inject(0.0){|sum,i| sum + (i-mu)**2 }/(x.length()-1)
  Math.sqrt(su) 
  #END YOUR CODE
end

:stdev

In [5]:
#Implement the error function given a weight vector, w
def dot x, w
  # BEGIN YOUR CODE
    x_name = x.keys
    w_name = w.keys

    i = 0
    dot = 0.0
    if x_name.length == 0 
      dot =  0.0
    else 
        while i<x_name.length do

          if w_name.include?x_name[i]
            index = w_name.index { |x| [x_name[i]].include?(x) }
            if x[x_name[i]].is_a? Numeric  and w[w_name[index]].is_a? Numeric
              dot += x[x_name[i]]*w[w_name[index]]
            else 
              dot +=0.0
            end
          else 
            dot += 0.0
          end
          i+=1
        end  
    end
  return dot
  #END YOUR CODE
end


:dot

In [6]:
def norm w
  # BEGIN YOUR CODE
  return Math.sqrt(dot w, w)
  #END YOUR CODE
end

:norm

In [7]:
# # BEGIN YOUR CODE
def cross_validate dataset, folds, &block
  examples = dataset["data"]
  # BEGIN YOUR CODE
  batch = examples.length/folds
  folds.times do |fold|


      test_data = dataset.clone
      test_data["data"] = test_data["data"][fold*batch,batch]  ##CV training examples

      train_data = dataset.clone
      train_data["data"] = train_data["data"] - test_data["data"]  ##CV testing examples

    ## Call the callback like this:
    yield train_data, test_data, fold
  end
  
  
  #END YOUR CODE
end


# #END YOUR CODE



:cross_validate

In [8]:
class ZScoreTransformer
  attr_reader :means, :stdevs
  
  def initialize feature_names
    @means = Hash.new
    @stdevs = Hash.new
    @feature_names = feature_names    
  end
  
  def train dataset
       
    examples = dataset["data"] 

    j = 0
    data = Array.new(@feature_names.length) { Array.new(examples.length) }
    while j < @feature_names.length do
          i = 0
          while i < examples.length do
            data[j][i]=(examples[i]["features"][@feature_names[j]])

            i +=1
          end
          
          m = mean data[j].reject { |e| e.nil? or e.is_a? String}
          s = stdev data[j].reject { |e| e.nil? or e.is_a? String}

          @means[@feature_names[j]] = m
          @stdevs[@feature_names[j]] = s

          j +=1
    end
    return @means,@stdevs
    # END YOUR CODE
  end
  
    def apply example_batch
    # BEGIN YOUR CODE

    i = 0
    feature =  (self.means).keys


    while i< feature.length do    

        if (self.means[feature[i]]).is_a? Numeric and   (self.stdevs[feature[i]]).is_a?Numeric and    (self.stdevs[feature[i]]) !=0
          j = 0
          while j<example_batch.length do
              if example_batch[j]["features"][feature[i]].is_a?Numeric
                example_batch[j]["features"][feature[i]] = (example_batch[j]["features"][feature[i]] - self.means[feature[i]])/self.stdevs[feature[i]]
              end
                j+=1
          end
        end
        i +=1
    end


    #END YOUR CODE
    return example_batch
    end
end

:apply

In [9]:
class MeanImputation
  attr_reader :means
  
  def initialize feature_names
    @means = Hash.new
    @feature_names = feature_names
  end
  
  def train dataset    
    # BEGIN YOUR CODE
    examples = dataset["data"] 

    j = 0
    data = Array.new(@feature_names.length) { Array.new(examples.length) }
    while j < @feature_names.length do
          i = 0
          while i < examples.length do
            data[j][i]=(examples[i]["features"][@feature_names[j]])

            i +=1
          end
          
          m = mean data[j].reject { |e| e.nil? or e.is_a? String}


          @means[@feature_names[j]] = m


          j +=1
          
    
    end

    return @means
    #END YOUR CODE
  end
  
  def apply(example_batch)
    # BEGIN YOUR CODE
    i = 0
    example = Hash.new()
    
    while i< example_batch.length do    
        
        j = 0
        name  = example_batch[i]["features"].keys
        while j < @feature_names.length do   

          if name.include?@feature_names[j]
             if example_batch[i]["features"][@feature_names[j]].is_a?Numeric
               example_batch[i]["features"][@feature_names[j]] = example_batch[i]["features"][@feature_names[j]]
             else
               example_batch[i]["features"][@feature_names[j]] = self.means[@feature_names[j]]
             end
          else 
             example_batch[i]["features"].store(@feature_names[j],self.means[@feature_names[j]])

          end
        j+=1
        end

        i +=1
    end
    #END YOUR CODE
    return example_batch
  end
end

:apply

In [10]:
class AgeRangeAsVector

  def initialize feature_name
    @feature_name = feature_name
  end
  def train dataset; end
  def apply(example_batch)
    min_age = 0
    max_age = 100
    pattern = "age_range_%d"
    # BEGIN YOUR CODE
    i = 0
     
    while i< example_batch.length do        
          if example_batch[i]["features"].keys.include? @feature_name
                bin = -example_batch[i]["features"][@feature_name]/(365*5)  *5

                example_batch[i]["features"].delete(@feature_name)

                binned_age = min_age

                while binned_age < max_age+1 do

                      new_feature_name = pattern% binned_age

                      if bin > binned_age or bin < (binned_age-5)    
                        if binned_age == max_age
                          example_batch[i]["features"].store(new_feature_name,1)
                        elsif binned_age == max_age
                          example_batch[i]["features"].store(new_feature_name,1)
                        else
                        example_batch[i]["features"].store(new_feature_name,nil)
                        end
                      else 
                        example_batch[i]["features"].store(new_feature_name,1)
                      end

                      binned_age+=5
                end
          else
          end
    i+=1
    end 

    
    #END YOUR CODE
    return example_batch
  end
end

:apply

In [11]:
class TargetAveraging
  attr_reader :means
  
  def initialize feature_names
    @means = Hash.new {|h,k| h[k] = Hash.new}
    @feature_names = feature_names
    @pattern = "avg_%s"
  end
  
  def train dataset    
    # BEGIN YOUR CODE
    i = 0
    while i< @feature_names.length do

          a = dataset.clone()["data"].select{|e|  !e["features"][@feature_names[i]].nil?}
          if a.empty?
          else
                b = a.uniq! {|e|  e["features"][@feature_names[i]] }
                name = b.map { |e|  e["features"][@feature_names[i]]  }           ######find unique value

                num =Array.new(name.length,0)
                lab_num = Array.new(name.length,0)
                j = 0
                while j< name.length do
                      a = dataset.clone()["data"].select{|e|  !e["features"][@feature_names[i]].nil?}
                      k = 0
                      while k< dataset["data"].length do
                            if dataset["data"][k]["features"][@feature_names[i]] == name[j]
                                  num[j]+=1
                                  lab_num[j] += dataset["data"][k]["label"]
                            end
                            k+=1
                      end

                      @means[@feature_names[i]].store(name[j],lab_num[j].to_f/num[j])
                      j+=1

                end
          end
          i+=1
    end
  
    return @means
    #END YOUR CODE
  end
  
    def apply(example_batch)
    # BEGIN YOUR CODE
    @pattern = "avg_%s"
    j = 0
    mean = self.means
    fea = mean.keys

    i = 0
    while i< fea.length do
      name = mean[fea[i]].keys
      j= 0
      
      while j<name.length do
        
        k = 0        
        while k < example_batch.length do

          if !(example_batch[k]["features"][fea[i]]).nil?
            value = example_batch[k]["features"][fea[i]]
            example_batch[k]["features"].delete(fea[i])

            example_batch[k]["features"].store(@pattern%fea[i],mean[fea[i]][value])
          end
          k+=1
        end
        
        j+=1
      end
      i+=1
    end

    
    
    #END YOUR CODE
    return example_batch
  end
  
end

:apply

In [12]:
class OneHotEncoding
  def initialize feature_names
    @feature_names = feature_names
    @pattern = "%s=%s"
  end
  
  def train dataset; end
  
  def apply(example_batch)
    
    # BEGIN YOUR CODE
    
    j = 0
    while j<example_batch.length do
      
          name = example_batch[j]["features"].keys
          i = 0
          while i< @feature_names.length do
              if name.include?@feature_names[i] and !(example_batch[j]["features"][@feature_names[i]]).nil?

                name_new = @pattern % [@feature_names[i], example_batch[j]["features"][@feature_names[i]]]
                example_batch[j]["features"].delete(@feature_names[i])
                example_batch[j]["features"].store(name_new,1.0)
              end
              i+=1
          end
          j+=1
    end
    
    #END YOUR CODE
    
    return example_batch
  end
end

:apply

In [13]:
class LogTransform
  def initialize feature_names
    @feature_names = feature_names
    @pattern = "log_%s"
  end
  
  def train dataset; end
  
  def apply(example_batch)
    # BEGIN YOUR CODE
    i = 0
    while i < @feature_names.length do
      j = 0
     
      while j<example_batch.length do
   
        if (example_batch[j]["features"].keys).include? @feature_names[i] and  example_batch[j]["features"][@feature_names[i]] > 0
                data = example_batch[j]["features"][@feature_names[i]]
                example_batch[j]["features"].delete(@feature_names[i])
                example_batch[j]["features"].store(@pattern % @feature_names[i], Math.log(data))
        end
        j+=1
      end
      
      
      i+=1
    end
    
    #END YOUR CODE
    return example_batch
  end
end

:apply

In [14]:
class L2Normalize
  def train dataset; end
  def apply(example_batch)
    # BEGIN YOUR CODE
      j = 0
      while j < example_batch.length do

          name = example_batch[j]["features"].keys
          i = 0
          normal_factor = norm example_batch[j]["features"]

          while i<name.length do
            if example_batch[j]["features"][name[i]].is_a? Numeric and normal_factor!=0
                example_batch[j]["features"][name[i]] = example_batch[j]["features"][name[i]]/normal_factor
            else
                example_batch[j]["features"][name[i]] = example_batch[j]["features"][name[i]]
            end
            i+=1
          end


          j+=1
      end    
    
    
    #END YOUR CODE
    return example_batch
  end
end

:apply

In [15]:
class FeatureTransformPipeline
  def initialize *transformers
    @transformers = transformers
  end
  
  def train dataset
    # BEGIN YOUR CODE 
    
    i = 0

    while i< @transformers.length do
        @transformers[i].train dataset 

        j = 0
        while j<dataset["data"].length do
  
          c = dataset["data"][j]
          dataset["data"][i] = (@transformers[i].apply  [c])[0]

          j+=1
        end
      i+=1
    end
       
    return dataset
    
    #END YOUR CODE
  end
  
  def apply example_batch 
    return @transformers.inject(example_batch) do |u, transform|
      u = transform.apply example_batch

    end
  end
end

:apply

In [16]:
# BEGIN YOUR CODE
class StochasticGradientDescent
  attr_reader :weights
  attr_reader :objective
  def initialize obj, w_0, lr 
    @objective = obj
    @weights = w_0
    @n = 1.0
    @lr = lr
  end
  def update x
    # BEGIN YOUR CODE

    lr  = @lr.to_f / Math.sqrt(@n)
    @n +=1

    if x.empty?
    else
      feature = x[0]["features"].keys
    
    end
  
    if (@weights.keys)==[0] and  x.length>0
      i=0
      while i<feature.length do
        @weights.store(feature[i],@weights[0])
        i+=1
      end
      
    end
    
    @weights.delete(0)
    
    i = 0
    gradient = @objective.grad x,@weights

    while i < @weights.length do
      if @weights[@weights.keys[i]].is_a?Numeric and gradient[@weights.keys[i]].is_a?Numeric
        @weights[@weights.keys[i]]  = @weights[@weights.keys[i]] - lr*gradient[@weights.keys[i]]        
      end
      i+=1
    end    

   return @objective.adjust @weights

  
    #END YOUR CODE
  end
end
#END YOUR CODE

:update

In [17]:
class LogisticRegressionModelL2
  def initialize reg_param
    @reg_param = reg_param
  end

  def predict row, w
    x = row["features"]    
    return 1.0 / (1 + Math.exp(-dot(w, x)))
  end
  
  def adjust w
    w.each_key {|k| w[k] = 0.0 if w[k].nan? or w[k].infinite?}
    w.each_key {|k| w[k] = 0.0 if w[k].abs > 1e5 }
  end
  
  def func data, w
    # BEGIN YOUR CODE
    adjust w
    i = 0
    sum  = 0.0

    while i< data.length do
      y=(data[i]["label"])
      yhat=dot(w, data[i]["features"])
      sum += Math.log(Math.exp(-y*yhat)+1.0)
      i+=1
    end

    
    return sum/data.length + 0.5 * @reg_param *(norm(w)**2)
  end 
   def grad data, w
    # BEGIN YOUR CODE
    
    g = Hash.new()
    key_list =[]
    data.each do |e|
        key_list.push((e["features"].keys))
    end
    name = key_list.flatten.uniq{|x| x  }

    sum  = Array.new(name.length)
    

    adjust w
    
    if (w.keys).empty?
      i = 0
      while i<name.length do
        w.store(name[i],w[0])
        i+=1
      end
      w.delete(0)
    end
    
    norm  = norm w

    i = 0
    while i< w.length do
      
      sum = 0.0
      j = 0
      while j<data.length
        yhat =dot  data[j]["features"], w
        ylabel = data[j]["label"]
        if data[j]["features"][name[i]].is_a? Numeric
          sum += (1.0/(1.0+Math.exp(ylabel*yhat)))*(-ylabel *data[j]["features"][name[i]] )
        end
        j+=1
      end
      sum = (sum/data.length) + @reg_param * w[name[i]]
      g.store(name[i],sum)
      i+=1
    end
    
    
    #END YOUR CODE
    return g
  end
  def adjust w
    w.each_key {|k| w[k] = 0.0 if w[k].nan? or w[k].infinite?}
    w.each_key {|k| w[k] = 0.0 if w[k].abs > 1e5 }
  end
end

:adjust

In [18]:
def num_negatives scores
  # BEGIN YOUR CODE

  return scores.length - num_positives(scores)
  
  #END YOUR CODE
end

:num_negatives

In [19]:
def num_positives scores
  # BEGIN YOUR CODE

  i = 0
  sum = 0
  while i< scores.length do
    if scores[i][1]>0
      sum += 1
    end
    i+=1
  end
  return sum
  
  #END YOUR CODE
end

:num_positives

In [20]:

class AUCMetric 
  include Metric  

  def roc_curve(scores)
  fp_rates = [0.0]
  tp_rates = [0.0]
  auc = 0.0

  # BEGIN YOUR CODE


  t_list = scores.sort_by{|e| [-e[0]]}
 
  i = 0
  total_pos = num_positives scores
  total_neg = num_negatives scores
  p = 0.0
  n = 0.0

  t_list.each do |e|

    if e[1] == 1
      p +=1.0   
      fp_rates.append(fp_rates[-1])      
      tp_rates.append(p/total_pos)
    else
      n +=1.0 
      fp_rates.append(n/total_neg)      
      tp_rates.append(tp_rates[-1])
    end

  end

  i = 1
  while i < (tp_rates.length) do
      auc+= 0.5*(fp_rates[i] -fp_rates[i-1])*(tp_rates[i] + tp_rates[i-1]  )
    i+=1
  end
  

  #END YOUR CODE
    return [fp_rates, tp_rates, auc]
  end



#   def calc_auc_only scores
#     fp, tp, auc = roc_curve scores
#     return auc
#   end
  def apply scores
    fp, tp, auc = roc_curve scores

    return auc
  end

  #END YOUR CODE
end

:apply

In [21]:
# BEGIN YOUR CODE
class LogisticRegressionLearner
  attr_reader :parameters
  attr_reader :weights  
  include Learner  

  def initialize regularization: 0.0, learning_rate: 0.01, batch_size: 20, epochs: 1
    @parameters = {"regularization" => regularization, 
      "learning_rate" => learning_rate, 
      "epochs" => epochs, "batch_size" => batch_size}
  end

  def train dataset


    reg = @parameters["regularization"]
    batch_size = @parameters["batch_size"]
    num_epoch = @parameters["epochs"]
    lr = @parameters["learning_rate"]

    w = @weights


    if w.nil?
      w = Hash.new {|h,k| h[k] =0.0}
    end

   losses = []

   num_epoch.times do

        obj = LogisticRegressionModelL2.new(reg)
        l = obj.func(dataset["data"][0,batch_size],w)
  
        losses.append(l)
        sgd = StochasticGradientDescent.new(obj, w, lr)
        sgd.update(dataset["data"][0,batch_size])

        w = sgd.weights

  
  end
  @weights = w
  
  return w
   
  end


  def predict example 
    
    x = example["features"]    
   
   return dot(@weights, x)
  end


  def evaluate dataset
   
    examples = dataset["data"]
    
    i = 0
    score = Array.new(examples.length) { Array.new(2) }
    while i<examples.length do      
      x = examples[i]   
      y = self.predict(x)   
      score[i][0] = y
      score[i][1] = x["label"]
      i+=1
    end

  return score
  end


end
#END YOUR CODE


:evaluate

In [22]:
## Add as many extra cells as you need

## Question 0.1
Use the cell below to create your ```ClassifierTwo``` which creates the training set. Assume that the evaluation dataset may come from a different evaluation database so don't just return the training set again. 

In [23]:
# , flag_own_car 
# , flag_own_realty
# , organization_type
# where ext_source_1 <> ''

In [24]:
class ClassifierTwo
  include FinalProjectClassifier  
  
  # BEGIN YOUR CODE
  include FinalProjectClassifier
#   include Learner
  
  
  # BEGIN YOUR CODE
  # Run SQL on the database and any transforms needed for the training database
  def create_training_dataset training_db
 sql =  "select target
, sk_id_curr
, ext_source_1
, ext_source_2
, ext_source_3
from application_train
where ext_source_1 <> ''
order by sk_id_curr"
 
    
#  sql =  "select target
# , sk_id_curr
# , amt_income_total
# , amt_credit
# , commonarea_avg
# , code_gender
# , name_family_status
# , name_housing_type
# , name_income_type
# , name_education_type 
# , organization_type
# , amt_annuity
# , days_employed
# , own_car_age
# , ext_source_1
# , ext_source_2
# , ext_source_3
# from application_train
# where sk_id_curr <> ''
# order by sk_id_curr" 
    
#     dataset2 = create_dataset training_db, sql  

#     dataset = Hash.new
#     dataset["features"] = dataset2["features"]
#     dataset["data"] = dataset2["data"][0,1000]

    
    dataset = create_dataset training_db, sql  
   
    
##transform 1    
    transform =  FeatureTransformPipeline.new(   
 
#     #Treat amt_income_total and amt_credit as log normal
#     LogTransform.new(%w(amt_income_total amt_credit)),
#     ZScoreTransformer.new(%w(log_amt_income_total log_amt_credit)),
#     MeanImputation.new(%w(log_amt_income_total log_amt_credit)), 
      
#     AgeRangeAsVector.new,     
#     OneHotEncoding.new(%w(name_family_status code_gender)),
      
 
#     TargetAveraging.new(%w(name_income_type flag_own_car flag_own_realty
#       name_family_status organization_type name_housing_type name_education_type)),      
#     L2Normalize.new
    ZScoreTransformer.new(%w( ext_source_1 ext_source_2 ext_source_3)),   
      
#     LogTransform.new(%w(amt_income_total amt_credit amt_annuity )),
#     ZScoreTransformer.new(%w(log_amt_income_total log_amt_credit amt_annuity 
# days_employed own_car_age  ext_source_1 ext_source_2 ext_source_3)),  
#     MeanImputation.new(%w(log_amt_income_total log_amt_credit days_employed own_car_age)), 
      

# #     OneHotEncoding.new(%w(name_family_status code_gender)),
      
 
#     TargetAveraging.new(%w(name_income_type name_family_status organization_type name_contract_type
#     name_housing_type name_education_type code_gender )),   
      


#     L2Normalize.new 
  )
##transform 2
    
    

    
    transform.train dataset
    dataset["data"].map {|x| x["bias"] = 1.0}

    
    return dataset
  end

  # Run SQL on the database and any transforms needed for the evaluation database
  def create_evaluation_dataset evaluation_db

    return create_training_dataset evaluation_db
  end

  #Return an array of learners
  
  def initialize min_auc: 0.48, max_auc: 0.6, folds: 5
    @parameters = {"min_auc" => min_auc, "max_auc"=> max_auc, "folds" => folds}
  end
  
  def create_learners dataset

    linear = LogisticRegressionLearner.new(regularization: 0.7, learning_rate: 0.7, batch_size: 128, epochs: 1)
    return [linear]
  end
  
  
  
  
  #END YOUR CODE
end


:create_learners

In [25]:
def test_75516f(test_data)
  test_basics test_data
end

test_data_75516f = {classifier: ClassifierTwo.new, min_auc: 0.6, max_auc: 1.0, folds: 5}
test_75516f(test_data_75516f)

Dev Set:
["ext_source_1", "ext_source_2", "ext_source_3"]
{"label"=>0, "id"=>456251, "features"=>{"ext_source_1"=>-1.6614269210261252, "ext_source_2"=>0.8465539163255636}, "bias"=>1.0}

Evaluation Set:
["ext_source_1", "ext_source_2", "ext_source_3"]
{"label"=>0, "id"=>456251, "features"=>{"ext_source_1"=>-1.6614269210261252, "ext_source_2"=>0.8465539163255636}, "bias"=>1.0}

Model trained on dev set:
LogisticRegressionLearner

Testing on
{"label"=>0, "id"=>136052, "features"=>{"ext_source_1"=>-1.3242580128759445, "ext_source_2"=>-1.2482188185534955, "ext_source_3"=>0.9422491642496317}, "bias"=>1.0}
0.007919222306940508


## Question 1.1 (5 points)

Validate that the classifier works by trying to create a small dataset and training the model on it. The model will be retrained later. This just verify that the interface is working.

In [26]:
def test_75516f(test_data)
  test_basics test_data
end

test_data_75516f = {classifier: ClassifierTwo.new, min_auc: 0.6, max_auc: 1.0, folds: 5}
test_75516f(test_data_75516f)

Dev Set:
["ext_source_1", "ext_source_2", "ext_source_3"]
{"label"=>0, "id"=>456251, "features"=>{"ext_source_1"=>-1.6614269210261252, "ext_source_2"=>0.8465539163255636}, "bias"=>1.0}

Evaluation Set:
["ext_source_1", "ext_source_2", "ext_source_3"]
{"label"=>0, "id"=>456251, "features"=>{"ext_source_1"=>-1.6614269210261252, "ext_source_2"=>0.8465539163255636}, "bias"=>1.0}

Model trained on dev set:
LogisticRegressionLearner

Testing on
{"label"=>0, "id"=>219957, "features"=>{"ext_source_1"=>0.8873429707865538, "ext_source_2"=>1.3311606222810841, "ext_source_3"=>0.9502662065655427}, "bias"=>1.0}
-0.04646368646651861


## Question 1.2 (15 points)

Measures the 5-fold cross-validation performance of your classifier. This may take a while, so be cognizant of the fact that your classifier may run out of memory--the server is a shared environment.

Notes:

1. This cell will timeout after 3 hours, after which you will see an ```Timeout::Error execution expired``` error. No exceptions.

In [27]:
### metrics.rb

#### KEEP THIS AT THE TOP OF YOUR FILE ####
def plot_roc_curve fp, tp, auc
  plot = Daru::DataFrame.new({x: fp, y: tp}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "False Positive Rate"
    plot.y_label "True Positive Rate"
    diagram.title("AUC: %.4f" % auc)
    plot.legend(true)
  end
end  

def cross_validation_model_performance dataset, folds, learners, metric    
  learners.map do |learner|
    tr_metrics = []
    te_metrics = []
    puts "#{folds}-fold CV: #{learner.class.name}, parameters: #{learner.parameters}"
    cross_validate dataset, folds do |train_dataset, test_dataset|
      learner.train train_dataset
      train_scores = learner.evaluate train_dataset
      test_scores = learner.evaluate test_dataset      
      tr_metrics << metric.apply(train_scores)
      te_metrics << metric.apply(test_scores)
    end
      
    #Train on full training set
    learner.train dataset
    learner_name = learner.name
    puts mean(te_metrics)
    {
      "learner" => learner_name, "trained_model" => learner, "parameters" => learner.parameters, "folds" => folds,
      "mean_train_metric" => mean(tr_metrics), "stdev_train_metric" => stdev(tr_metrics),
      "mean_test_metric" => mean(te_metrics), "stdev_test_metric" => stdev(te_metrics),
    }
  end
end

def best_performance_by_learner stats  
  stats.group_by {|s| s["learner"]}.map do |g_s|
    learner, learner_stats = g_s
    best_parameters = learner_stats.max_by {|l| l["mean_test_metric"]}    
    [learner, best_parameters]
  end.to_h
end

def parameter_search learners, dataset, folds = 5
  metric = AUCMetric.new  
  stats = cross_validation_model_performance dataset, folds, learners, metric
  best_by_learner = best_performance_by_learner stats  
    summary = Hash.new
    best_by_learner.each_key do |k|
        summary[k] = best_by_learner[k].clone
        summary[k].delete "trained_model"
    end
  puts JSON.pretty_generate(summary)

  assert_equal learners.size, stats.size
  assert_true(stats.all? {|s| a = s["mean_train_metric"]; a >= 0.0 and a <= 1.0}, "0 <= Train AUC <= 1")
  assert_true(stats.all? {|s| a = s["mean_test_metric"]; a >= 0.0 and a <= 1.0}, "0 <= Train AUC <= 1")
  
  stats.map! {|s| t = s.clone; t.delete "trained_model"; t}
  df = Daru::DataFrame.new(stats) 
    
  return [df, best_by_learner]
end

### ADD YOUR CODE AFTER THIS LINE ###

:parameter_search

In [ ]:
def test_9e9453(test_data)
  run_cross_validation_performance test_data
  test_data[:cross_validation_results]
end
test_9e9453(test_data_75516f)

Creating training dataset
Creating learners
Running 5-fold cross validation
5-fold CV: LogisticRegressionLearner, parameters: {"regularization"=>0.7, "learning_rate"=>0.7, "epochs"=>1, "batch_size"=>128}


In [ ]:
def test_758950(test_data)
  test_cross_validation_performance test_data
end
test_758950(test_data_75516f)

## Question 1.3 (20 points)

Plots the ROC curve on the dev dataset, which is a separate database. Checks that the AUC is within target. Note that this assumes the model has already been trained.

In [ ]:
def test_f67f50(test_data)
  test_data[:db] = dev_db()
  test_data[:db_size] = 15334
  test_data[:name] = "dev_eval"
  test_evaluation_set_performance test_data
end
test_f67f50(test_data_75516f)

Creating evaluation dataset
Evaluating classifier
[[456251, 0.0], [100007, 0.0], [100015, 0.0], [100025, 0.0], [100068, 0.0]]
Validating predictions against labels from database


ArgumentError: There is no prediction for 100002. Make sure you are not removing any records.

## Question 1.3 (60 Points)

Tests your model on the test dataset. This is a hidden test, so you will not see this until after you submit. 

In [370]:


def test_2eca1f(test_data)
  hidden_test_data = test_data.clone
end
test_2eca1f(test_data_75516f)
test_data_75516f = nil

## Notes

We will use all the models, feature extraction, and evaluation methods you learned in this course on a realistic dataset. In the final project, you will create better and better classifiers for the final project dataset. In each set of classifiers, create a dataset directly from the database. Assume that the database will be provided to you and **do not assume** that the database will be the same all the time.

Note that the test set performance will be a **HIDDEN** test. You may find that your model does not perform as well on the hidden test as on the training set. To mitigate this, we are using cross-validation and a hold-out set which is about the same size as the testing set. 

Note the following requirements:
1. Focus on data preparation and creating, normalizing, understanding new features.
1. You may use R notebooks on this server for exploration but anything you obtain there must run in this notebook
1. Use your own implementation of models, which you will have refactored as part of [Final Project Tests](./final_project_tests.ipynb)
1. Avoid creating new models that we did not cover in the class. There is no credit for fancy models.
1. Do not use the target label, or anything that is derived based on the training label as features. 
1. You may talk to other students about your solution, but do not share code. This is an individual assignment.


For this and all subsequent questions, create your own class following the ```FinalProjectClassifier``` interface below.

Each example in the dataset you create will have an ```id``` field. This is how we will track performance. Please note that that you must provide a score for each example in the evaluation database provided to your classifier. So please take care to keep all examples in the evaluation set. You may sample the training set as much as you want. 

The scores will extracted from the database and your ```AUCMetric``` will be applied.

Here is an example of a classifier:
```ruby
class ClassifierOne
  include FinalProjectClassifier
  
  # Run SQL on the database and any transforms needed for the training database
  def create_training_dataset training_db
    sql = <<SQL
select sk_id_curr, target
from application_train  
order by sk_id_curr
SQL
    dataset = create_dataset training_db, sql    
    dataset["data"].map {|x| x["bias"] = 1.0}
    return dataset
  end
  
  # Run SQL on the database and any transforms needed for the evaluation database
  def create_evaluation_dataset evaluation_db
    return create_training_dataset evaluation_db
  end
  
  #Return an array of learners
  def create_learners dataset
    linear = LogisticRegressionLearner.new(regularization: 0.01, learning_rate: 0.001, batch_size: 20, epochs: 1)
    return [linear]
  end

end
```

